### Pre-processing the 19 Camden High Streets into Survival Format

In [1]:
# import necessary libraries
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from requests import get
from urllib.parse import urlparse
from datetime import datetime

In [2]:
%cd '/home/jovyan/work/Dissertation/Camden High Streets'

/home/jovyan/work/Dissertation/Camden High Streets


In [3]:
os.getcwd()

'/home/jovyan/work/Dissertation/Camden High Streets'

### Read in the high street subsets

In [4]:
Brecknock = pd.read_csv("Brecknock-points.csv", low_memory=False)

In [5]:
Kentish = pd.read_csv("Kentish-points.csv", low_memory=False)

In [6]:
Fortress = pd.read_csv("Fortress-points.csv", low_memory=False)

In [7]:
CamdenT = pd.read_csv("CamdenT-points.csv", low_memory=False)

In [8]:
Belsize = pd.read_csv("Belsize-points.csv", low_memory=False)

In [9]:
Loudoun = pd.read_csv("Loudoun-points.csv", low_memory=False)

In [10]:
Haverstock = pd.read_csv("Haverstock-points.csv", low_memory=False)

In [11]:
SouthEnd = pd.read_csv("SouthEnd-points.csv", low_memory=False)

In [12]:
#Highgate = pd.read_csv("Highgate-points.csv", low_memory=False)

In [13]:
Cricklewood = pd.read_csv("Cricklewood-points.csv", low_memory=False)

In [14]:
Regents = pd.read_csv("Regents-points.csv", low_memory=False)

In [15]:
Queens = pd.read_csv("Queens-points.csv", low_memory=False)

In [16]:
Kilburn = pd.read_csv("Kilburn-points.csv", low_memory=False)

In [17]:
Mill = pd.read_csv("Mill-points.csv", low_memory=False)

In [18]:
Fortune = pd.read_csv("Fortune-points.csv", low_memory=False)

In [19]:
Willesden = pd.read_csv("Willesden-points.csv", low_memory=False)

In [20]:
#Finchley = pd.read_csv("Finchley-points.csv", low_memory=False)

In [21]:
Mansfield = pd.read_csv("Mansfield-points.csv", low_memory=False)

In [22]:
WestEnd = pd.read_csv("WestEnd-points.csv", low_memory=False)

In [23]:
Hampstead = pd.read_csv("Hampstead-points.csv", low_memory=False)

In [24]:
Eversholt = pd.read_csv("Eversholt-points.csv", low_memory=False)

### Run the preprocessing template for each dataset 

#### 1. Brecknock 

In [25]:
Brecknock = Brecknock.sort_values(by= ['status_date'])

In [26]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [27]:
#deleted those cols columns from the dataset as they only contain nulls

Brecknock = Brecknock.drop(cols, axis=1)

In [28]:
Brecknock['new_date'] = pd.to_datetime(Brecknock['status_date'], errors='coerce')

In [29]:
Brecknock = Brecknock.sort_values(by= ['new_date'])

In [30]:
Brecknock['new_date'].loc[(Brecknock['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [31]:
Brecknock = Brecknock.sort_values(by= ['location_code', 'period'])

In [32]:
Brecknock['locaton_code'] = Brecknock['location_code'].astype("category")

In [33]:
locations = Brecknock['location_code'].unique()

In [34]:
Brecknock.loc[Brecknock['name'].isnull(), 'name'] = Brecknock['location_code']

In [35]:
Brecknock['status_date'] = pd.to_datetime(Brecknock['status_date'], errors='coerce')

In [36]:
Brecknock['status_date'].iloc[Brecknock['status_date'] < '2010-04-01'] = '04/01/2010'

In [37]:
Brecknock['status_date'] = pd.to_datetime(Brecknock['status_date'], errors='coerce')

In [38]:
Brecknock['period2'] = pd.to_datetime(Brecknock['period'])

In [39]:
Brecknock['period2'] = Brecknock['period2'] + pd.DateOffset(months=1)

In [40]:
Brecknock['duration'] = Brecknock['period2'] - Brecknock['status_date']

In [41]:
Brecknock['duration'] = Brecknock['duration'].dt.days

In [42]:
Brecknock['study-start'] = '04/01/2010'

In [43]:
Brecknock['study-start'] = pd.to_datetime(Brecknock['study-start'], errors='coerce')

In [44]:
Brecknock['time-of-entry'] = Brecknock['status_date'] -  Brecknock['study-start']

In [45]:
Brecknock['time-at-open'] = 0

In [46]:
Brecknock = Brecknock.drop(Brecknock[(Brecknock['status'] == True)].index)

In [47]:
Brecknock['location_code'] = Brecknock['location_code'].astype("category")

In [48]:
locations = list(Brecknock['location_code'].unique())

In [49]:
Brecknock = Brecknock.set_index('location_code')

In [50]:
Brecknock['survival'] = 0

In [51]:
Brecknock['survival'] = Brecknock['duration']

In [52]:
Brecknock['entry'] = Brecknock['time-of-entry'].dt.days

In [53]:
Brecknock['time-at-open'] = Brecknock['survival'] + Brecknock['entry']

In [54]:
Brecknock['stat'] = Brecknock['status'].astype(int)

In [55]:
Brecknock["StatusChange"] =  Brecknock['stat'].shift() != Brecknock['stat']

In [56]:
Brecknock["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [57]:
Brecknock["TenantChange"] =  Brecknock['name'].shift(-1) != Brecknock['name'] 

In [58]:
Brecknock["TenantChange"].iloc[0] = False

In [59]:
counter = 0 
stoppingCondition = len(locations)
Brecknock2 = pd.DataFrame(data=None, columns=Brecknock.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Brecknock.loc[i, :].iloc[[ Brecknock.loc[i, :]['status_duration'].argmax()]]
        Brecknock2 = Brecknock2.append(m, ignore_index=False)
        counter += 1

In [60]:
Brecknock3 = Brecknock2.drop_duplicates()

In [61]:
Brecknock3['highstreet'] = 'Brecknock'

In [62]:
Brecknock3.to_csv('Brecknock-vacant-survival.csv')

#### 2. Kentish 

In [63]:
Kentish = Kentish.sort_values(by= ['status_date'])

In [64]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [65]:
Kentish

,Unnamed: 0,Unnamed: 0.1,rates_code,rates_authority_id,rates_authority,region,location_code,address_no,address_street,address_town,...,V69,V70,V71,V72,V73,V74,V75,V76,V77,geometry
3880,329295,329295,51201460012,E09000007,Camden,Inner London - West,8079315000,GND - 2ND FLRS 146,ROYAL COLLEGE STREET,LONDON,...,D,NaN,E05000143,NaN,2013-07-01,0,CO,203.0,NaN,POINT (529304.7813367147 184070.2960136788)
4399,402250,402250,6.54002E+11,E09000007,Camden,Inner London - West,11659715000,UNIT 4 AT 21,BONNY STREET,LONDON,...,C,NaN,E05000130,NaN,NaN,0,CO,203.0,NaN,POINT (529149.7842568328 184183.290960255)
962,82795,82795,5.1202E+11,E09000007,Camden,Inner London - West,75247063,201,ROYAL COLLEGE STREET,LONDON,...,D,NaN,E05000130,NaN,2012-03-19,0,CS,249.0,NaN,POINT (529229.7802553794 184107.2885801889)
3481,123894,123894,0074200631009A,E09000007,Camden,Inner London - West,95659063,UNIT 7 AT 63,PRATT STREET,LONDON,...,L,NaN,E05000143,NaN,NaN,0,IF3,96.0,NaN,POINT (529377.7846075087 183993.2942823111)
2997,113205,113205,6.57089E+11,E09000007,Camden,Inner London - West,3251736000,1 LAWFORDS WHARF,LYME STREET,LONDON,...,D,NaN,E05000130,NaN,2012-03-19,0,CO,203.0,NaN,POINT (529185.7852216109 184064.2957215998)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,85831,85831,5.35006E+11,E09000007,Camden,Inner London - West,99271063,GND F 57,CAMDEN ROAD,LONDON,...,D,NaN,E05000131,NaN,2012-03-19,3,CS2,24.0,NaN,POINT (529196.7824470752 184232.3021180349)
3250,113502,113502,6.61009E+11,E09000007,Camden,Inner London - West,110608063,ARCHES 90 & 91,BAYNES STREET,LONDON,...,D,NaN,E05000131,NaN,2012-03-19,0,CG1,289.0,NaN,POINT (529354.7875236059 184154.2926299943)
874,82707,82707,5.12019E+11,E09000007,Camden,Inner London - West,3386993000,2ND F 187-191,ROYAL COLLEGE STREET,LONDON,...,C,NaN,E05000130,NaN,NaN,0,CO,203.0,NaN,POINT (529258.7831391912 184058.2975567151)
3426,123839,123839,0074200631007B,E09000007,Camden,Inner London - West,95657063,UNIT 5 AT 63,PRATT STREET,LONDON,...,D,NaN,E05000143,NaN,2013-10-22,0,IF3,96.0,NaN,POINT (529337.7843255643 183958.291385453)


In [66]:
#deleted those cols columns from the dataset as they only contain nulls

Kentish = Kentish.drop(cols, axis=1)

In [67]:
Kentish['new_date'] = pd.to_datetime(Kentish['status_date'], errors='coerce')

In [68]:
Kentish = Kentish.sort_values(by= ['new_date'])

In [69]:
Kentish['new_date'].loc[(Kentish['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [70]:
Kentish = Kentish.sort_values(by= ['location_code', 'period'])

In [71]:
Kentish['locaton_code'] = Kentish['location_code'].astype("category")

In [72]:
locations = Kentish['location_code'].unique()

In [73]:
Kentish.loc[Kentish['name'].isnull(), 'name'] = Kentish['location_code']

In [74]:
Kentish['status_date'] = pd.to_datetime(Kentish['status_date'], errors='coerce')

In [75]:
Kentish['status_date'].iloc[Kentish['status_date'] < '2010-04-01'] = '04/01/2010'

In [76]:
Kentish['status_date'] = pd.to_datetime(Kentish['status_date'], errors='coerce')

In [77]:
Kentish['period2'] = pd.to_datetime(Kentish['period'])

In [78]:
Kentish['period2'] = Kentish['period2'] + pd.DateOffset(months=1)

In [79]:
Kentish['duration'] = Kentish['period2'] - Kentish['status_date']

In [80]:
Kentish['duration'] = Kentish['duration'].dt.days

In [81]:
Kentish['study-start'] = '04/01/2010'

In [82]:
Kentish['study-start'] = pd.to_datetime(Kentish['study-start'], errors='coerce')

In [83]:
Kentish['time-of-entry'] = Kentish['status_date'] -  Kentish['study-start']

In [84]:
Kentish['time-at-open'] = 0

In [ ]:
Kentish['vacant'] = 0

In [ ]:
Kentish = Kentish.drop(Kentish[(Kentish['status'] == True)].index)

In [ ]:
Kentish['location_code'] = Kentish['location_code'].astype("category")

In [ ]:
locations = list(Kentish['location_code'].unique())

In [ ]:
Kentish = Kentish.set_index('location_code')

In [ ]:
Kentish['survival'] = 0

In [ ]:
Kentish['survival'] = Kentish['duration']

In [ ]:
Kentish['entry'] = Kentish['time-of-entry'].dt.days

In [ ]:
Kentish['time-at-open'] = Kentish['survival'] + Kentish['entry']

In [ ]:
Kentish['stat'] = Kentish['status'].astype(int)

In [ ]:
Kentish["StatusChange"] =  Kentish['stat'].shift() != Kentish['stat']

In [ ]:
Kentish["StatusChange"].iloc[0] = False

In [ ]:
Kentish["TenantChange"] =  Kentish['name'].shift(-1) != Kentish['name'] 

In [ ]:
Kentish["TenantChange"].iloc[0] = False

In [ ]:
counter = 0 
stoppingCondition = len(locations)
Kentish2 = pd.DataFrame(data=None, columns=Kentish.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Kentish.loc[i, :].iloc[[ Kentish.loc[i, :]['status_duration'].argmax()]]
        Kentish2 = Kentish2.append(m, ignore_index=False)
        counter += 1

In [ ]:
Kentish3 = Kentish2.drop_duplicates()

In [ ]:
Kentish3['highstreet'] = 'Kentish'

In [ ]:
Kentish3.to_csv('Kentish-vacant-survival.csv')

#### 3. Fortress 

In [148]:
Fortress = Fortress.sort_values(by= ['status_date'])

In [149]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [150]:
#deleted those cols columns from the dataset as they only contain nulls

Fortress = Fortress.drop(cols, axis=1)

In [151]:
Fortress['new_date'] = pd.to_datetime(Fortress['status_date'], errors='coerce')

In [152]:
Fortress = Fortress.sort_values(by= ['new_date'])

In [153]:
Fortress['new_date'].loc[(Fortress['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [154]:
Fortress = Fortress.sort_values(by= ['location_code', 'period'])

In [155]:
Fortress['locaton_code'] = Fortress['location_code'].astype("category")

In [156]:
locations = Fortress['location_code'].unique()

In [157]:
Fortress.loc[Fortress['name'].isnull(), 'name'] = Fortress['location_code']

In [158]:
Fortress['status_date'] = pd.to_datetime(Fortress['status_date'], errors='coerce')

In [159]:
Fortress['status_date'].iloc[Fortress['status_date'] < '2010-04-01'] = '04/01/2010'

In [160]:
Fortress['status_date'] = pd.to_datetime(Fortress['status_date'], errors='coerce')

In [161]:
Fortress['period2'] = pd.to_datetime(Fortress['period'])

In [162]:
Fortress['period2'] = Fortress['period2'] + pd.DateOffset(months=1)

In [163]:
Fortress['duration'] = Fortress['period2'] - Fortress['status_date']

In [164]:
Fortress['duration'] = Fortress['duration'].dt.days

In [165]:
Fortress['study-start'] = '04/01/2010'

In [166]:
Fortress['study-start'] = pd.to_datetime(Fortress['study-start'], errors='coerce')

In [167]:
Fortress['time-of-entry'] = Fortress['status_date'] -  Fortress['study-start']

In [168]:
Fortress['time-at-open'] = 0

In [169]:
Fortress['vacant'] = 0

In [170]:
Fortress = Fortress.drop(Fortress[(Fortress['status'] == True)].index)

In [171]:
Fortress['location_code'] = Fortress['location_code'].astype("category")

In [172]:
locations = list(Fortress['location_code'].unique())

In [173]:
Fortress = Fortress.set_index('location_code')

In [174]:
Fortress['survival'] = 0

In [175]:
Fortress['survival'] = Fortress['duration']

In [176]:
Fortress['entry'] = Fortress['time-of-entry'].dt.days

In [177]:
Fortress['time-at-open'] = Fortress['survival'] + Fortress['entry']

In [178]:
Fortress['stat'] = Fortress['status'].astype(int)

In [179]:
Fortress["StatusChange"] =  Fortress['stat'].shift() != Fortress['stat']

In [180]:
Fortress["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [181]:
Fortress["TenantChange"] =  Fortress['name'].shift(-1) != Fortress['name'] 

In [182]:
Fortress["TenantChange"].iloc[0] = False

In [184]:
counter = 0 
stoppingCondition = len(locations)
Fortress2 = pd.DataFrame(data=None, columns=Fortress.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Fortress.loc[i, :].iloc[[ Fortress.loc[i, :]['status_duration'].argmax()]]
        Fortress2 = Fortress2.append(m, ignore_index=False)
        counter += 1

In [186]:
Fortress3 = Fortress2.drop_duplicates()

In [188]:
Fortress3['highstreet'] = 'Fortress'

In [189]:
Fortress3.to_csv('Fortress-vacant-survival.csv')

#### 4. Camden Town

In [190]:
CamdenT = CamdenT.sort_values(by= ['status_date'])

In [191]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [192]:
#deleted those cols columns from the dataset as they only contain nulls

CamdenT = CamdenT.drop(cols, axis=1)

In [193]:
CamdenT['new_date'] = pd.to_datetime(CamdenT['status_date'], errors='coerce')

In [194]:
CamdenT = CamdenT.sort_values(by= ['new_date'])

In [195]:
CamdenT['new_date'].loc[(CamdenT['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [196]:
CamdenT = CamdenT.sort_values(by= ['location_code', 'period'])

In [197]:
CamdenT['locaton_code'] = CamdenT['location_code'].astype("category")

In [198]:
locations = CamdenT['location_code'].unique()

In [199]:
CamdenT.loc[CamdenT['name'].isnull(), 'name'] = CamdenT['location_code']

In [200]:
CamdenT['status_date'] = pd.to_datetime(CamdenT['status_date'], errors='coerce')

In [201]:
CamdenT['status_date'].iloc[CamdenT['status_date'] < '2010-04-01'] = '04/01/2010'

In [202]:
CamdenT['status_date'] = pd.to_datetime(CamdenT['status_date'], errors='coerce')

In [203]:
CamdenT['period2'] = pd.to_datetime(CamdenT['period'])

In [204]:
CamdenT['period2'] = CamdenT['period2'] + pd.DateOffset(months=1)

In [205]:
CamdenT['duration'] = CamdenT['period2'] - CamdenT['status_date']

In [206]:
CamdenT['duration'] = CamdenT['duration'].dt.days

In [207]:
CamdenT['study-start'] = '04/01/2010'

In [208]:
CamdenT['study-start'] = pd.to_datetime(CamdenT['study-start'], errors='coerce')

In [209]:
CamdenT['time-of-entry'] = CamdenT['status_date'] -  CamdenT['study-start']

In [210]:
CamdenT['time-at-open'] = 0

In [211]:
CamdenT['vacant'] = 0

In [212]:
CamdenT = CamdenT.drop(CamdenT[(CamdenT['status'] == True)].index)

In [213]:
CamdenT['location_code'] = CamdenT['location_code'].astype("category")

In [214]:
locations = list(CamdenT['location_code'].unique())

In [215]:
CamdenT = CamdenT.set_index('location_code')

In [216]:
CamdenT['survival'] = 0

In [217]:
CamdenT['survival'] = CamdenT['duration']

In [218]:
CamdenT['entry'] = CamdenT['time-of-entry'].dt.days

In [219]:
CamdenT['time-at-open'] = CamdenT['survival'] + CamdenT['entry']

In [220]:
CamdenT['stat'] = CamdenT['status'].astype(int)

In [221]:
CamdenT["StatusChange"] =  CamdenT['stat'].shift() != CamdenT['stat']

In [222]:
CamdenT["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [223]:
CamdenT["TenantChange"] =  CamdenT['name'].shift(-1) != CamdenT['name'] 

In [224]:
CamdenT["TenantChange"].iloc[0] = False

In [226]:
counter = 0 
stoppingCondition = len(locations)
CamdenT2 = pd.DataFrame(data=None, columns=CamdenT.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = CamdenT.loc[i, :].iloc[[ CamdenT.loc[i, :]['status_duration'].argmax()]]
        CamdenT2 = CamdenT2.append(m, ignore_index=False)
        counter += 1

In [228]:
CamdenT3 = CamdenT2.drop_duplicates()

In [230]:
CamdenT3['highstreet'] = 'CamdenT'

In [231]:
CamdenT3.to_csv('CamdenT-vacant-survival.csv')

#### 5. Belsize

In [234]:
Belsize = Belsize.sort_values(by= ['status_date'])

In [235]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [236]:
#deleted those cols columns from the dataset as they only contain nulls

Belsize = Belsize.drop(cols, axis=1)

In [237]:
Belsize['new_date'] = pd.to_datetime(Belsize['status_date'], errors='coerce')

In [238]:
Belsize = Belsize.sort_values(by= ['new_date'])

In [239]:
Belsize['new_date'].loc[(Belsize['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [240]:
Belsize = Belsize.sort_values(by= ['location_code', 'period'])

In [241]:
Belsize['locaton_code'] = Belsize['location_code'].astype("category")

In [242]:
locations = Belsize['location_code'].unique()

In [243]:
Belsize.loc[Belsize['name'].isnull(), 'name'] = Belsize['location_code']

In [244]:
Belsize['status_date'] = pd.to_datetime(Belsize['status_date'], errors='coerce')

In [245]:
Belsize['status_date'].iloc[Belsize['status_date'] < '2010-04-01'] = '04/01/2010'

In [246]:
Belsize['status_date'] = pd.to_datetime(Belsize['status_date'], errors='coerce')

In [247]:
Belsize['period2'] = pd.to_datetime(Belsize['period'])

In [248]:
Belsize['period2'] = Belsize['period2'] + pd.DateOffset(months=1)

In [249]:
Belsize['duration'] = Belsize['period2'] - Belsize['status_date']

In [250]:
Belsize['duration'] = Belsize['duration'].dt.days

In [251]:
Belsize['study-start'] = '04/01/2010'

In [252]:
Belsize['study-start'] = pd.to_datetime(Belsize['study-start'], errors='coerce')

In [253]:
Belsize['time-of-entry'] = Belsize['status_date'] -  Belsize['study-start']

In [254]:
Belsize['time-at-open'] = 0

In [255]:
Belsize['vacant'] = 0

In [256]:
Belsize = Belsize.drop(Belsize[(Belsize['status'] == True)].index)

In [257]:
Belsize['location_code'] = Belsize['location_code'].astype("category")

In [258]:
locations = list(Belsize['location_code'].unique())

In [259]:
Belsize = Belsize.set_index('location_code')

In [260]:
Belsize['survival'] = 0

In [261]:
Belsize['survival'] = Belsize['duration']

In [262]:
Belsize['entry'] = Belsize['time-of-entry'].dt.days

In [263]:
Belsize['time-at-open'] = Belsize['survival'] + Belsize['entry']

In [264]:
Belsize['stat'] = Belsize['status'].astype(int)

In [265]:
Belsize["StatusChange"] =  Belsize['stat'].shift() != Belsize['stat']

In [266]:
Belsize["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [267]:
Belsize["TenantChange"] =  Belsize['name'].shift(-1) != Belsize['name'] 

In [268]:
Belsize["TenantChange"].iloc[0] = False

In [270]:
counter = 0 
stoppingCondition = len(locations)
Belsize2 = pd.DataFrame(data=None, columns=Belsize.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Belsize.loc[i, :].iloc[[ Belsize.loc[i, :]['status_duration'].argmax()]]
        Belsize2 = Belsize2.append(m, ignore_index=False)
        counter += 1

In [272]:
Belsize3 = Belsize2.drop_duplicates()

In [274]:
Belsize3['highstreet'] = 'Belsize'

In [275]:
Belsize3.to_csv('Belsize-vacant-survival.csv')

#### 6. Loudoun

In [276]:
Loudoun = Loudoun.sort_values(by= ['status_date'])

In [277]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [278]:
#deleted those cols columns from the dataset as they only contain nulls

Loudoun = Loudoun.drop(cols, axis=1)

In [279]:
Loudoun['new_date'] = pd.to_datetime(Loudoun['status_date'], errors='coerce')

In [280]:
Loudoun = Loudoun.sort_values(by= ['new_date'])

In [281]:
Loudoun['new_date'].loc[(Loudoun['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [282]:
Loudoun = Loudoun.sort_values(by= ['location_code', 'period'])

In [283]:
Loudoun['locaton_code'] = Loudoun['location_code'].astype("category")

In [284]:
locations = Loudoun['location_code'].unique()

In [285]:
Loudoun.loc[Loudoun['name'].isnull(), 'name'] = Loudoun['location_code']

In [286]:
Loudoun['status_date'] = pd.to_datetime(Loudoun['status_date'], errors='coerce')

In [287]:
Loudoun['status_date'].iloc[Loudoun['status_date'] < '2010-04-01'] = '04/01/2010'

In [288]:
Loudoun['status_date'] = pd.to_datetime(Loudoun['status_date'], errors='coerce')

In [289]:
Loudoun['period2'] = pd.to_datetime(Loudoun['period'])

In [290]:
Loudoun['period2'] = Loudoun['period2'] + pd.DateOffset(months=1)

In [291]:
Loudoun['duration'] = Loudoun['period2'] - Loudoun['status_date']

In [292]:
Loudoun['duration'] = Loudoun['duration'].dt.days

In [293]:
Loudoun['study-start'] = '04/01/2010'

In [294]:
Loudoun['study-start'] = pd.to_datetime(Loudoun['study-start'], errors='coerce')

In [295]:
Loudoun['time-of-entry'] = Loudoun['status_date'] -  Loudoun['study-start']

In [296]:
Loudoun['time-at-open'] = 0

In [297]:
Loudoun['vacant'] = 0

In [298]:
Loudoun = Loudoun.drop(Loudoun[(Loudoun['status'] == True)].index)

In [299]:
Loudoun['location_code'] = Loudoun['location_code'].astype("category")

In [300]:
locations = list(Loudoun['location_code'].unique())

In [301]:
Loudoun = Loudoun.set_index('location_code')

In [302]:
Loudoun['survival'] = 0

In [303]:
Loudoun['survival'] = Loudoun['duration']

In [304]:
Loudoun['entry'] = Loudoun['time-of-entry'].dt.days

In [305]:
Loudoun['time-at-open'] = Loudoun['survival'] + Loudoun['entry']

In [306]:
Loudoun['stat'] = Loudoun['status'].astype(int)

In [307]:
Loudoun["StatusChange"] =  Loudoun['stat'].shift() != Loudoun['stat']

In [308]:
Loudoun["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [309]:
Loudoun["TenantChange"] =  Loudoun['name'].shift(-1) != Loudoun['name'] 

In [310]:
Loudoun["TenantChange"].iloc[0] = False

In [312]:
counter = 0 
stoppingCondition = len(locations)
Loudoun2 = pd.DataFrame(data=None, columns=Loudoun.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Loudoun.loc[i, :].iloc[[ Loudoun.loc[i, :]['status_duration'].argmax()]]
        Loudoun2 = Loudoun2.append(m, ignore_index=False)
        counter += 1

In [314]:
Loudoun3 = Loudoun2.drop_duplicates()

In [316]:
Loudoun3['highstreet'] = 'Loudoun'

In [317]:
Loudoun3.to_csv('Loudoun-vacant-survival.csv')

#### 7. Haverstock

In [318]:
Haverstock = Haverstock.sort_values(by= ['status_date'])

In [319]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [320]:
#deleted those cols columns from the dataset as they only contain nulls

Haverstock = Haverstock.drop(cols, axis=1)

In [321]:
Haverstock['new_date'] = pd.to_datetime(Haverstock['status_date'], errors='coerce')

In [322]:
Haverstock = Haverstock.sort_values(by= ['new_date'])

In [323]:
Haverstock['new_date'].loc[(Haverstock['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [324]:
Haverstock = Haverstock.sort_values(by= ['location_code', 'period'])

In [325]:
Haverstock['locaton_code'] = Haverstock['location_code'].astype("category")

In [326]:
locations = Haverstock['location_code'].unique()

In [327]:
Haverstock.loc[Haverstock['name'].isnull(), 'name'] = Haverstock['location_code']

In [328]:
Haverstock['status_date'] = pd.to_datetime(Haverstock['status_date'], errors='coerce')

In [329]:
Haverstock['status_date'].iloc[Haverstock['status_date'] < '2010-04-01'] = '04/01/2010'

In [330]:
Haverstock['status_date'] = pd.to_datetime(Haverstock['status_date'], errors='coerce')

In [331]:
Haverstock['period2'] = pd.to_datetime(Haverstock['period'])

In [332]:
Haverstock['period2'] = Haverstock['period2'] + pd.DateOffset(months=1)

In [333]:
Haverstock['duration'] = Haverstock['period2'] - Haverstock['status_date']

In [334]:
Haverstock['duration'] = Haverstock['duration'].dt.days

In [335]:
Haverstock['study-start'] = '04/01/2010'

In [336]:
Haverstock['study-start'] = pd.to_datetime(Haverstock['study-start'], errors='coerce')

In [337]:
Haverstock['time-of-entry'] = Haverstock['status_date'] -  Haverstock['study-start']

In [338]:
Haverstock['time-at-open'] = 0

In [339]:
Haverstock['vacant'] = 0

In [340]:
Haverstock = Haverstock.drop(Haverstock[(Haverstock['status'] == True)].index)

In [341]:
Haverstock['location_code'] = Haverstock['location_code'].astype("category")

In [342]:
locations = list(Haverstock['location_code'].unique())

In [343]:
Haverstock = Haverstock.set_index('location_code')

In [344]:
Haverstock['survival'] = 0

In [345]:
Haverstock['survival'] = Haverstock['duration']

In [346]:
Haverstock['entry'] = Haverstock['time-of-entry'].dt.days

In [347]:
Haverstock['time-at-open'] = Haverstock['survival'] + Haverstock['entry']

In [348]:
Haverstock['stat'] = Haverstock['status'].astype(int)

In [349]:
Haverstock["StatusChange"] =  Haverstock['stat'].shift() != Haverstock['stat']

In [350]:
Haverstock["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [351]:
Haverstock["TenantChange"] =  Haverstock['name'].shift(-1) != Haverstock['name'] 

In [352]:
Haverstock["TenantChange"].iloc[0] = False

In [354]:
counter = 0 
stoppingCondition = len(locations)
Haverstock2 = pd.DataFrame(data=None, columns=Haverstock.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Haverstock.loc[i, :].iloc[[ Haverstock.loc[i, :]['status_duration'].argmax()]]
        Haverstock2 = Haverstock2.append(m, ignore_index=False)
        counter += 1

In [356]:
Haverstock3 = Haverstock2.drop_duplicates()

In [358]:
Haverstock3['highstreet'] = 'Haverstock'

In [359]:
Haverstock3.to_csv('Haverstock-vacant-survival.csv')

#### 8. SouthEnd

In [360]:
SouthEnd = SouthEnd.sort_values(by= ['status_date'])

In [361]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [362]:
#deleted those cols columns from the dataset as they only contain nulls

SouthEnd = SouthEnd.drop(cols, axis=1)

In [363]:
SouthEnd['new_date'] = pd.to_datetime(SouthEnd['status_date'], errors='coerce')

In [364]:
SouthEnd = SouthEnd.sort_values(by= ['new_date'])

In [365]:
SouthEnd['new_date'].loc[(SouthEnd['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [366]:
SouthEnd = SouthEnd.sort_values(by= ['location_code', 'period'])

In [367]:
SouthEnd['locaton_code'] = SouthEnd['location_code'].astype("category")

In [368]:
locations = SouthEnd['location_code'].unique()

In [369]:
SouthEnd.loc[SouthEnd['name'].isnull(), 'name'] = SouthEnd['location_code']

In [370]:
SouthEnd['status_date'] = pd.to_datetime(SouthEnd['status_date'], errors='coerce')

In [371]:
SouthEnd['status_date'].iloc[SouthEnd['status_date'] < '2010-04-01'] = '04/01/2010'

In [372]:
SouthEnd['status_date'] = pd.to_datetime(SouthEnd['status_date'], errors='coerce')

In [373]:
SouthEnd['period2'] = pd.to_datetime(SouthEnd['period'])

In [374]:
SouthEnd['period2'] = SouthEnd['period2'] + pd.DateOffset(months=1)

In [375]:
SouthEnd['duration'] = SouthEnd['period2'] - SouthEnd['status_date']

In [376]:
SouthEnd['duration'] = SouthEnd['duration'].dt.days

In [377]:
SouthEnd['study-start'] = '04/01/2010'

In [378]:
SouthEnd['study-start'] = pd.to_datetime(SouthEnd['study-start'], errors='coerce')

In [379]:
SouthEnd['time-of-entry'] = SouthEnd['status_date'] -  SouthEnd['study-start']

In [380]:
SouthEnd['time-at-open'] = 0

In [381]:
SouthEnd['vacant'] = 0

In [382]:
SouthEnd = SouthEnd.drop(SouthEnd[(SouthEnd['status'] == True)].index)

In [383]:
SouthEnd['location_code'] = SouthEnd['location_code'].astype("category")

In [384]:
locations = list(SouthEnd['location_code'].unique())

In [385]:
SouthEnd = SouthEnd.set_index('location_code')

In [386]:
SouthEnd['survival'] = 0

In [387]:
SouthEnd['survival'] = SouthEnd['duration']

In [388]:
SouthEnd['entry'] = SouthEnd['time-of-entry'].dt.days

In [389]:
SouthEnd['time-at-open'] = SouthEnd['survival'] + SouthEnd['entry']

In [390]:
SouthEnd['stat'] = SouthEnd['status'].astype(int)

In [391]:
SouthEnd["StatusChange"] =  SouthEnd['stat'].shift() != SouthEnd['stat']

In [392]:
SouthEnd["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [393]:
SouthEnd["TenantChange"] =  SouthEnd['name'].shift(-1) != SouthEnd['name'] 

In [394]:
SouthEnd["TenantChange"].iloc[0] = False

In [396]:
counter = 0 
stoppingCondition = len(locations)
SouthEnd2 = pd.DataFrame(data=None, columns=SouthEnd.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = SouthEnd.loc[i, :].iloc[[ SouthEnd.loc[i, :]['status_duration'].argmax()]]
        SouthEnd2 = SouthEnd2.append(m, ignore_index=False)
        counter += 1

In [398]:
SouthEnd3 = SouthEnd2.drop_duplicates()

In [400]:
SouthEnd3['highstreet'] = 'SouthEnd'

In [401]:
SouthEnd3.to_csv('SouthEnd-vacant-survival.csv')

#### 9. Cricklewood

In [402]:
Cricklewood = Cricklewood.sort_values(by= ['status_date'])

In [403]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [404]:
#deleted those cols columns from the dataset as they only contain nulls

Cricklewood = Cricklewood.drop(cols, axis=1)

In [405]:
Cricklewood['new_date'] = pd.to_datetime(Cricklewood['status_date'], errors='coerce')

In [406]:
Cricklewood = Cricklewood.sort_values(by= ['new_date'])

In [407]:
Cricklewood['new_date'].loc[(Cricklewood['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [408]:
Cricklewood = Cricklewood.sort_values(by= ['location_code', 'period'])

In [409]:
Cricklewood['locaton_code'] = Cricklewood['location_code'].astype("category")

In [410]:
locations = Cricklewood['location_code'].unique()

In [411]:
Cricklewood.loc[Cricklewood['name'].isnull(), 'name'] = Cricklewood['location_code']

In [412]:
Cricklewood['status_date'] = pd.to_datetime(Cricklewood['status_date'], errors='coerce')

In [413]:
Cricklewood['status_date'].iloc[Cricklewood['status_date'] < '2010-04-01'] = '04/01/2010'

In [414]:
Cricklewood['status_date'] = pd.to_datetime(Cricklewood['status_date'], errors='coerce')

In [415]:
Cricklewood['period2'] = pd.to_datetime(Cricklewood['period'])

In [416]:
Cricklewood['period2'] = Cricklewood['period2'] + pd.DateOffset(months=1)

In [417]:
Cricklewood['duration'] = Cricklewood['period2'] - Cricklewood['status_date']

In [418]:
Cricklewood['duration'] = Cricklewood['duration'].dt.days

In [419]:
Cricklewood['study-start'] = '04/01/2010'

In [420]:
Cricklewood['study-start'] = pd.to_datetime(Cricklewood['study-start'], errors='coerce')

In [421]:
Cricklewood['time-of-entry'] = Cricklewood['status_date'] -  Cricklewood['study-start']

In [422]:
Cricklewood['time-at-open'] = 0

In [423]:
Cricklewood['vacant'] = 0

In [424]:
Cricklewood = Cricklewood.drop(Cricklewood[(Cricklewood['status'] == False) & (Cricklewood['status_date'] == '2010-04-01')].index)

In [425]:
Cricklewood['location_code'] = Cricklewood['location_code'].astype("category")

In [426]:
locations = list(Cricklewood['location_code'].unique())

In [427]:
Cricklewood = Cricklewood.set_index('location_code')

In [428]:
Cricklewood['survival'] = 0

In [429]:
Cricklewood['survival'] = Cricklewood['duration']

In [430]:
Cricklewood['entry'] = Cricklewood['time-of-entry'].dt.days

In [431]:
Cricklewood['time-at-open'] = Cricklewood['survival'] + Cricklewood['entry']

In [432]:
Cricklewood['stat'] = Cricklewood['status'].astype(int)

In [433]:
Cricklewood["StatusChange"] =  Cricklewood['stat'].shift() != Cricklewood['stat']

In [434]:
Cricklewood["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [435]:
Cricklewood["TenantChange"] =  Cricklewood['name'].shift(-1) != Cricklewood['name'] 

In [436]:
Cricklewood["TenantChange"].iloc[0] = False

In [437]:
CricklewoodSub = Cricklewood[(Cricklewood['StatusChange'] == False)  & (Cricklewood['TenantChange'] == True) & (Cricklewood['status'] == True)]

In [438]:
counter = 0 
stoppingCondition = len(locations)
Cricklewood2 = pd.DataFrame(data=None, columns=Cricklewood.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Cricklewood.loc[i, :].iloc[[ Cricklewood.loc[i, :]['status_duration'].argmax()]]
        Cricklewood2 = Cricklewood2.append(m, ignore_index=False)
        counter += 1

In [439]:
Cricklewood3 = CricklewoodSub.append(Cricklewood2)

In [440]:
Cricklewood3 = Cricklewood3.drop_duplicates()

In [441]:
Cricklewood3.loc[Cricklewood3['time-at-open'] < 4018, ['vacant']] = 1

In [442]:
Cricklewood3['highstreet'] = 'Cricklewood'

In [443]:
Cricklewood3.to_csv('Cricklewood-vacant-survival.csv')

#### 10. Regents

In [444]:
Regents = Regents.sort_values(by= ['status_date'])

In [445]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [446]:
#deleted those cols columns from the dataset as they only contain nulls

Regents = Regents.drop(cols, axis=1)

In [447]:
Regents['new_date'] = pd.to_datetime(Regents['status_date'], errors='coerce')

In [448]:
Regents = Regents.sort_values(by= ['new_date'])

In [449]:
Regents['new_date'].loc[(Regents['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [450]:
Regents = Regents.sort_values(by= ['location_code', 'period'])

In [451]:
Regents['locaton_code'] = Regents['location_code'].astype("category")

In [452]:
locations = Regents['location_code'].unique()

In [453]:
Regents.loc[Regents['name'].isnull(), 'name'] = Regents['location_code']

In [454]:
Regents['status_date'] = pd.to_datetime(Regents['status_date'], errors='coerce')

In [455]:
Regents['status_date'].iloc[Regents['status_date'] < '2010-04-01'] = '04/01/2010'

In [456]:
Regents['status_date'] = pd.to_datetime(Regents['status_date'], errors='coerce')

In [457]:
Regents['period2'] = pd.to_datetime(Regents['period'])

In [458]:
Regents['period2'] = Regents['period2'] + pd.DateOffset(months=1)

In [459]:
Regents['duration'] = Regents['period2'] - Regents['status_date']

In [460]:
Regents['duration'] = Regents['duration'].dt.days

In [461]:
Regents['study-start'] = '04/01/2010'

In [462]:
Regents['study-start'] = pd.to_datetime(Regents['study-start'], errors='coerce')

In [463]:
Regents['time-of-entry'] = Regents['status_date'] -  Regents['study-start']

In [464]:
Regents['time-at-open'] = 0

In [465]:
Regents['vacant'] = 0

In [466]:
Regents = Regents.drop(Regents[(Regents['status'] == False) & (Regents['status_date'] == '2010-04-01')].index)

In [467]:
Regents['location_code'] = Regents['location_code'].astype("category")

In [468]:
locations = list(Regents['location_code'].unique())

In [469]:
Regents = Regents.set_index('location_code')

In [470]:
Regents['survival'] = 0

In [471]:
Regents['survival'] = Regents['duration']

In [472]:
Regents['entry'] = Regents['time-of-entry'].dt.days

In [473]:
Regents['time-at-open'] = Regents['survival'] + Regents['entry']

In [474]:
Regents['stat'] = Regents['status'].astype(int)

In [475]:
Regents["StatusChange"] =  Regents['stat'].shift() != Regents['stat']

In [476]:
Regents["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [477]:
Regents["TenantChange"] =  Regents['name'].shift(-1) != Regents['name'] 

In [478]:
Regents["TenantChange"].iloc[0] = False

In [479]:
RegentsSub = Regents[(Regents['StatusChange'] == False)  & (Regents['TenantChange'] == True) & (Regents['status'] == True)]

In [480]:
counter = 0 
stoppingCondition = len(locations)
Regents2 = pd.DataFrame(data=None, columns=Regents.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Regents.loc[i, :].iloc[[ Regents.loc[i, :]['status_duration'].argmax()]]
        Regents2 = Regents2.append(m, ignore_index=False)
        counter += 1

In [481]:
Regents3 = RegentsSub.append(Regents2)

In [482]:
Regents3 = Regents3.drop_duplicates()

In [483]:
Regents3.loc[Regents3['time-at-open'] < 4018, ['vacant']] = 1

In [484]:
Regents3['highstreet'] = 'Regents'

In [485]:
Regents3.to_csv('Regents-vacant-survival.csv')

#### 11. Queens

In [486]:
Queens = Queens.sort_values(by= ['status_date'])

In [487]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [488]:
#deleted those cols columns from the dataset as they only contain nulls

Queens = Queens.drop(cols, axis=1)

In [489]:
Queens['new_date'] = pd.to_datetime(Queens['status_date'], errors='coerce')

In [490]:
Queens = Queens.sort_values(by= ['new_date'])

In [491]:
Queens['new_date'].loc[(Queens['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [492]:
Queens = Queens.sort_values(by= ['location_code', 'period'])

In [493]:
Queens['locaton_code'] = Queens['location_code'].astype("category")

In [494]:
locations = Queens['location_code'].unique()

In [495]:
Queens.loc[Queens['name'].isnull(), 'name'] = Queens['location_code']

In [496]:
Queens['status_date'] = pd.to_datetime(Queens['status_date'], errors='coerce')

In [497]:
Queens['status_date'].iloc[Queens['status_date'] < '2010-04-01'] = '04/01/2010'

In [498]:
Queens['status_date'] = pd.to_datetime(Queens['status_date'], errors='coerce')

In [499]:
Queens['period2'] = pd.to_datetime(Queens['period'])

In [500]:
Queens['period2'] = Queens['period2'] + pd.DateOffset(months=1)

In [501]:
Queens['duration'] = Queens['period2'] - Queens['status_date']

In [502]:
Queens['duration'] = Queens['duration'].dt.days

In [503]:
Queens['study-start'] = '04/01/2010'

In [504]:
Queens['study-start'] = pd.to_datetime(Queens['study-start'], errors='coerce')

In [505]:
Queens['time-of-entry'] = Queens['status_date'] -  Queens['study-start']

In [506]:
Queens['time-at-open'] = 0

In [507]:
Queens['vacant'] = 0

In [508]:
Queens = Queens.drop(Queens[(Queens['status'] == False) & (Queens['status_date'] == '2010-04-01')].index)

In [509]:
Queens['location_code'] = Queens['location_code'].astype("category")

In [510]:
locations = list(Queens['location_code'].unique())

In [511]:
Queens = Queens.set_index('location_code')

In [512]:
Queens['survival'] = 0

In [513]:
Queens['survival'] = Queens['duration']

In [514]:
Queens['entry'] = Queens['time-of-entry'].dt.days

In [515]:
Queens['time-at-open'] = Queens['survival'] + Queens['entry']

In [516]:
Queens['stat'] = Queens['status'].astype(int)

In [517]:
Queens["StatusChange"] =  Queens['stat'].shift() != Queens['stat']

In [518]:
Queens["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [519]:
Queens["TenantChange"] =  Queens['name'].shift(-1) != Queens['name'] 

In [520]:
Queens["TenantChange"].iloc[0] = False

In [521]:
QueensSub = Queens[(Queens['StatusChange'] == False)  & (Queens['TenantChange'] == True) & (Queens['status'] == True)]

In [522]:
counter = 0 
stoppingCondition = len(locations)
Queens2 = pd.DataFrame(data=None, columns=Queens.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Queens.loc[i, :].iloc[[ Queens.loc[i, :]['status_duration'].argmax()]]
        Queens2 = Queens2.append(m, ignore_index=False)
        counter += 1

In [523]:
Queens3 = QueensSub.append(Queens2)

In [524]:
Queens3 = Queens3.drop_duplicates()

In [525]:
Queens3.loc[Queens3['time-at-open'] < 4018, ['vacant']] = 1

In [526]:
Queens3['highstreet'] = 'Queens'

In [527]:
Queens3.to_csv('Queens-vacant-survival.csv')

#### 12. Kilburn

In [528]:
Kilburn = Kilburn.sort_values(by= ['status_date'])

In [529]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [530]:
#deleted those cols columns from the dataset as they only contain nulls

Kilburn = Kilburn.drop(cols, axis=1)

In [531]:
Kilburn['new_date'] = pd.to_datetime(Kilburn['status_date'], errors='coerce')

In [532]:
Kilburn = Kilburn.sort_values(by= ['new_date'])

In [533]:
Kilburn['new_date'].loc[(Kilburn['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [534]:
Kilburn = Kilburn.sort_values(by= ['location_code', 'period'])

In [535]:
Kilburn['locaton_code'] = Kilburn['location_code'].astype("category")

In [536]:
locations = Kilburn['location_code'].unique()

In [537]:
Kilburn.loc[Kilburn['name'].isnull(), 'name'] = Kilburn['location_code']

In [538]:
Kilburn['status_date'] = pd.to_datetime(Kilburn['status_date'], errors='coerce')

In [539]:
Kilburn['status_date'].iloc[Kilburn['status_date'] < '2010-04-01'] = '04/01/2010'

In [540]:
Kilburn['status_date'] = pd.to_datetime(Kilburn['status_date'], errors='coerce')

In [541]:
Kilburn['period2'] = pd.to_datetime(Kilburn['period'])

In [542]:
Kilburn['period2'] = Kilburn['period2'] + pd.DateOffset(months=1)

In [543]:
Kilburn['duration'] = Kilburn['period2'] - Kilburn['status_date']

In [544]:
Kilburn['duration'] = Kilburn['duration'].dt.days

In [545]:
Kilburn['study-start'] = '04/01/2010'

In [546]:
Kilburn['study-start'] = pd.to_datetime(Kilburn['study-start'], errors='coerce')

In [547]:
Kilburn['time-of-entry'] = Kilburn['status_date'] -  Kilburn['study-start']

In [548]:
Kilburn['time-at-open'] = 0

In [549]:
Kilburn['vacant'] = 0

In [550]:
Kilburn = Kilburn.drop(Kilburn[(Kilburn['status'] == False) & (Kilburn['status_date'] == '2010-04-01')].index)

In [551]:
Kilburn['location_code'] = Kilburn['location_code'].astype("category")

In [552]:
locations = list(Kilburn['location_code'].unique())

In [553]:
Kilburn = Kilburn.set_index('location_code')

In [554]:
Kilburn['survival'] = 0

In [555]:
Kilburn['survival'] = Kilburn['duration']

In [556]:
Kilburn['entry'] = Kilburn['time-of-entry'].dt.days

In [557]:
Kilburn['time-at-open'] = Kilburn['survival'] + Kilburn['entry']

In [558]:
Kilburn['stat'] = Kilburn['status'].astype(int)

In [559]:
Kilburn["StatusChange"] =  Kilburn['stat'].shift() != Kilburn['stat']

In [560]:
Kilburn["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [561]:
Kilburn["TenantChange"] =  Kilburn['name'].shift(-1) != Kilburn['name'] 

In [562]:
Kilburn["TenantChange"].iloc[0] = False

In [563]:
KilburnSub = Kilburn[(Kilburn['StatusChange'] == False)  & (Kilburn['TenantChange'] == True) & (Kilburn['status'] == True)]

In [564]:
counter = 0 
stoppingCondition = len(locations)
Kilburn2 = pd.DataFrame(data=None, columns=Kilburn.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Kilburn.loc[i, :].iloc[[ Kilburn.loc[i, :]['status_duration'].argmax()]]
        Kilburn2 = Kilburn2.append(m, ignore_index=False)
        counter += 1

In [565]:
Kilburn3 = KilburnSub.append(Kilburn2)

In [566]:
Kilburn3 = Kilburn3.drop_duplicates()

In [567]:
Kilburn3.loc[Kilburn3['time-at-open'] < 4018, ['vacant']] = 1

In [568]:
Kilburn3['highstreet'] = 'Kilburn'

In [569]:
Kilburn3.to_csv('Kilburn-vacant-survival.csv')

#### 13. Mill

In [570]:
Mill = Mill.sort_values(by= ['status_date'])

In [571]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [572]:
#deleted those cols columns from the dataset as they only contain nulls

Mill = Mill.drop(cols, axis=1)

In [573]:
Mill['new_date'] = pd.to_datetime(Mill['status_date'], errors='coerce')

In [574]:
Mill = Mill.sort_values(by= ['new_date'])

In [575]:
Mill['new_date'].loc[(Mill['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [576]:
Mill = Mill.sort_values(by= ['location_code', 'period'])

In [577]:
Mill['locaton_code'] = Mill['location_code'].astype("category")

In [578]:
locations = Mill['location_code'].unique()

In [579]:
Mill.loc[Mill['name'].isnull(), 'name'] = Mill['location_code']

In [580]:
Mill['status_date'] = pd.to_datetime(Mill['status_date'], errors='coerce')

In [581]:
Mill['status_date'].iloc[Mill['status_date'] < '2010-04-01'] = '04/01/2010'

In [582]:
Mill['status_date'] = pd.to_datetime(Mill['status_date'], errors='coerce')

In [583]:
Mill['period2'] = pd.to_datetime(Mill['period'])

In [584]:
Mill['period2'] = Mill['period2'] + pd.DateOffset(months=1)

In [585]:
Mill['duration'] = Mill['period2'] - Mill['status_date']

In [586]:
Mill['duration'] = Mill['duration'].dt.days

In [587]:
Mill['study-start'] = '04/01/2010'

In [588]:
Mill['study-start'] = pd.to_datetime(Mill['study-start'], errors='coerce')

In [589]:
Mill['time-of-entry'] = Mill['status_date'] -  Mill['study-start']

In [590]:
Mill['time-at-open'] = 0

In [591]:
Mill['vacant'] = 0

In [592]:
Mill = Mill.drop(Mill[(Mill['status'] == False) & (Mill['status_date'] == '2010-04-01')].index)

In [593]:
Mill['location_code'] = Mill['location_code'].astype("category")

In [594]:
locations = list(Mill['location_code'].unique())

In [595]:
Mill = Mill.set_index('location_code')

In [596]:
Mill['survival'] = 0

In [597]:
Mill['survival'] = Mill['duration']

In [598]:
Mill['entry'] = Mill['time-of-entry'].dt.days

In [599]:
Mill['time-at-open'] = Mill['survival'] + Mill['entry']

In [600]:
Mill['stat'] = Mill['status'].astype(int)

In [601]:
Mill["StatusChange"] =  Mill['stat'].shift() != Mill['stat']

In [602]:
Mill["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [603]:
Mill["TenantChange"] =  Mill['name'].shift(-1) != Mill['name'] 

In [604]:
Mill["TenantChange"].iloc[0] = False

In [605]:
MillSub = Mill[(Mill['StatusChange'] == False)  & (Mill['TenantChange'] == True) & (Mill['status'] == True)]

In [606]:
counter = 0 
stoppingCondition = len(locations)
Mill2 = pd.DataFrame(data=None, columns=Mill.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Mill.loc[i, :].iloc[[ Mill.loc[i, :]['status_duration'].argmax()]]
        Mill2 = Mill2.append(m, ignore_index=False)
        counter += 1

In [607]:
Mill3 = MillSub.append(Mill2)

In [608]:
Mill3 = Mill3.drop_duplicates()

In [609]:
Mill3.loc[Mill3['time-at-open'] < 4018, ['vacant']] = 1

In [610]:
Mill3['highstreet'] = 'Mill'

In [611]:
Mill3.to_csv('Mill-vacant-survival.csv')

#### 14. Fortune

In [612]:
Fortune = Fortune.sort_values(by= ['status_date'])

In [613]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [614]:
#deleted those cols columns from the dataset as they only contain nulls

Fortune = Fortune.drop(cols, axis=1)

In [615]:
Fortune['new_date'] = pd.to_datetime(Fortune['status_date'], errors='coerce')

In [616]:
Fortune = Fortune.sort_values(by= ['new_date'])

In [617]:
Fortune['new_date'].loc[(Fortune['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [618]:
Fortune = Fortune.sort_values(by= ['location_code', 'period'])

In [619]:
Fortune['locaton_code'] = Fortune['location_code'].astype("category")

In [620]:
locations = Fortune['location_code'].unique()

In [621]:
Fortune.loc[Fortune['name'].isnull(), 'name'] = Fortune['location_code']

In [622]:
Fortune['status_date'] = pd.to_datetime(Fortune['status_date'], errors='coerce')

In [623]:
Fortune['status_date'].iloc[Fortune['status_date'] < '2010-04-01'] = '04/01/2010'

In [624]:
Fortune['status_date'] = pd.to_datetime(Fortune['status_date'], errors='coerce')

In [625]:
Fortune['period2'] = pd.to_datetime(Fortune['period'])

In [626]:
Fortune['period2'] = Fortune['period2'] + pd.DateOffset(months=1)

In [627]:
Fortune['duration'] = Fortune['period2'] - Fortune['status_date']

In [628]:
Fortune['duration'] = Fortune['duration'].dt.days

In [629]:
Fortune['study-start'] = '04/01/2010'

In [630]:
Fortune['study-start'] = pd.to_datetime(Fortune['study-start'], errors='coerce')

In [631]:
Fortune['time-of-entry'] = Fortune['status_date'] -  Fortune['study-start']

In [632]:
Fortune['time-at-open'] = 0

In [633]:
Fortune['vacant'] = 0

In [634]:
Fortune = Fortune.drop(Fortune[(Fortune['status'] == False) & (Fortune['status_date'] == '2010-04-01')].index)

In [635]:
Fortune['location_code'] = Fortune['location_code'].astype("category")

In [636]:
locations = list(Fortune['location_code'].unique())

In [637]:
Fortune = Fortune.set_index('location_code')

In [638]:
Fortune['survival'] = 0

In [639]:
Fortune['survival'] = Fortune['duration']

In [640]:
Fortune['entry'] = Fortune['time-of-entry'].dt.days

In [641]:
Fortune['time-at-open'] = Fortune['survival'] + Fortune['entry']

In [642]:
Fortune['stat'] = Fortune['status'].astype(int)

In [643]:
Fortune["StatusChange"] =  Fortune['stat'].shift() != Fortune['stat']

In [644]:
Fortune["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [645]:
Fortune["TenantChange"] =  Fortune['name'].shift(-1) != Fortune['name'] 

In [646]:
Fortune["TenantChange"].iloc[0] = False

In [647]:
FortuneSub = Fortune[(Fortune['StatusChange'] == False)  & (Fortune['TenantChange'] == True) & (Fortune['status'] == True)]

In [648]:
counter = 0 
stoppingCondition = len(locations)
Fortune2 = pd.DataFrame(data=None, columns=Fortune.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Fortune.loc[i, :].iloc[[ Fortune.loc[i, :]['status_duration'].argmax()]]
        Fortune2 = Fortune2.append(m, ignore_index=False)
        counter += 1

In [649]:
Fortune3 = FortuneSub.append(Fortune2)

In [650]:
Fortune3 = Fortune3.drop_duplicates()

In [651]:
Fortune3.loc[Fortune3['time-at-open'] < 4018, ['vacant']] = 1

In [652]:
Fortune3['highstreet'] = 'Fortune'

In [653]:
Fortune3.to_csv('Fortune-vacant-survival.csv')

#### 15. Willesden

In [654]:
Willesden = Willesden.sort_values(by= ['status_date'])

In [655]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [656]:
#deleted those cols columns from the dataset as they only contain nulls

Willesden = Willesden.drop(cols, axis=1)

In [657]:
Willesden['new_date'] = pd.to_datetime(Willesden['status_date'], errors='coerce')

In [658]:
Willesden = Willesden.sort_values(by= ['new_date'])

In [659]:
Willesden['new_date'].loc[(Willesden['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [660]:
Willesden = Willesden.sort_values(by= ['location_code', 'period'])

In [661]:
Willesden['locaton_code'] = Willesden['location_code'].astype("category")

In [662]:
locations = Willesden['location_code'].unique()

In [663]:
Willesden.loc[Willesden['name'].isnull(), 'name'] = Willesden['location_code']

In [664]:
Willesden['status_date'] = pd.to_datetime(Willesden['status_date'], errors='coerce')

In [665]:
Willesden['status_date'].iloc[Willesden['status_date'] < '2010-04-01'] = '04/01/2010'

In [666]:
Willesden['status_date'] = pd.to_datetime(Willesden['status_date'], errors='coerce')

In [667]:
Willesden['period2'] = pd.to_datetime(Willesden['period'])

In [668]:
Willesden['period2'] = Willesden['period2'] + pd.DateOffset(months=1)

In [669]:
Willesden['duration'] = Willesden['period2'] - Willesden['status_date']

In [670]:
Willesden['duration'] = Willesden['duration'].dt.days

In [671]:
Willesden['study-start'] = '04/01/2010'

In [672]:
Willesden['study-start'] = pd.to_datetime(Willesden['study-start'], errors='coerce')

In [673]:
Willesden['time-of-entry'] = Willesden['status_date'] -  Willesden['study-start']

In [674]:
Willesden['time-at-open'] = 0

In [675]:
Willesden['vacant'] = 0

In [676]:
Willesden = Willesden.drop(Willesden[(Willesden['status'] == False) & (Willesden['status_date'] == '2010-04-01')].index)

In [677]:
Willesden['location_code'] = Willesden['location_code'].astype("category")

In [678]:
locations = list(Willesden['location_code'].unique())

In [679]:
Willesden = Willesden.set_index('location_code')

In [680]:
Willesden['survival'] = 0

In [681]:
Willesden['survival'] = Willesden['duration']

In [682]:
Willesden['entry'] = Willesden['time-of-entry'].dt.days

In [683]:
Willesden['time-at-open'] = Willesden['survival'] + Willesden['entry']

In [684]:
Willesden['stat'] = Willesden['status'].astype(int)

In [685]:
Willesden["StatusChange"] =  Willesden['stat'].shift() != Willesden['stat']

In [686]:
Willesden["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [687]:
Willesden["TenantChange"] =  Willesden['name'].shift(-1) != Willesden['name'] 

In [688]:
Willesden["TenantChange"].iloc[0] = False

In [689]:
WillesdenSub = Willesden[(Willesden['StatusChange'] == False)  & (Willesden['TenantChange'] == True) & (Willesden['status'] == True)]

In [690]:
counter = 0 
stoppingCondition = len(locations)
Willesden2 = pd.DataFrame(data=None, columns=Willesden.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Willesden.loc[i, :].iloc[[ Willesden.loc[i, :]['status_duration'].argmax()]]
        Willesden2 = Willesden2.append(m, ignore_index=False)
        counter += 1

In [691]:
Willesden3 = WillesdenSub.append(Willesden2)

In [692]:
Willesden3 = Willesden3.drop_duplicates()

In [693]:
Willesden3.loc[Willesden3['time-at-open'] < 4018, ['vacant']] = 1

In [694]:
Willesden3['highstreet'] = 'Willesden'

In [695]:
Willesden3.to_csv('Willesden-vacant-survival.csv')

#### 16. Mansfield

In [696]:
Mansfield = Mansfield.sort_values(by= ['status_date'])

In [697]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [698]:
#deleted those cols columns from the dataset as they only contain nulls

Mansfield = Mansfield.drop(cols, axis=1)

In [699]:
Mansfield['new_date'] = pd.to_datetime(Mansfield['status_date'], errors='coerce')

In [700]:
Mansfield = Mansfield.sort_values(by= ['new_date'])

In [701]:
Mansfield['new_date'].loc[(Mansfield['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [702]:
Mansfield = Mansfield.sort_values(by= ['location_code', 'period'])

In [703]:
Mansfield['locaton_code'] = Mansfield['location_code'].astype("category")

In [704]:
locations = Mansfield['location_code'].unique()

In [705]:
Mansfield.loc[Mansfield['name'].isnull(), 'name'] = Mansfield['location_code']

In [706]:
Mansfield['status_date'] = pd.to_datetime(Mansfield['status_date'], errors='coerce')

In [707]:
Mansfield['status_date'].iloc[Mansfield['status_date'] < '2010-04-01'] = '04/01/2010'

In [708]:
Mansfield['status_date'] = pd.to_datetime(Mansfield['status_date'], errors='coerce')

In [709]:
Mansfield['period2'] = pd.to_datetime(Mansfield['period'])

In [710]:
Mansfield['period2'] = Mansfield['period2'] + pd.DateOffset(months=1)

In [711]:
Mansfield['duration'] = Mansfield['period2'] - Mansfield['status_date']

In [712]:
Mansfield['duration'] = Mansfield['duration'].dt.days

In [713]:
Mansfield['study-start'] = '04/01/2010'

In [714]:
Mansfield['study-start'] = pd.to_datetime(Mansfield['study-start'], errors='coerce')

In [715]:
Mansfield['time-of-entry'] = Mansfield['status_date'] -  Mansfield['study-start']

In [716]:
Mansfield['time-at-open'] = 0

In [717]:
Mansfield['vacant'] = 0

In [718]:
Mansfield = Mansfield.drop(Mansfield[(Mansfield['status'] == False) & (Mansfield['status_date'] == '2010-04-01')].index)

In [719]:
Mansfield['location_code'] = Mansfield['location_code'].astype("category")

In [720]:
locations = list(Mansfield['location_code'].unique())

In [721]:
Mansfield = Mansfield.set_index('location_code')

In [722]:
Mansfield['survival'] = 0

In [723]:
Mansfield['survival'] = Mansfield['duration']

In [724]:
Mansfield['entry'] = Mansfield['time-of-entry'].dt.days

In [725]:
Mansfield['time-at-open'] = Mansfield['survival'] + Mansfield['entry']

In [726]:
Mansfield['stat'] = Mansfield['status'].astype(int)

In [727]:
Mansfield["StatusChange"] =  Mansfield['stat'].shift() != Mansfield['stat']

In [728]:
Mansfield["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [729]:
Mansfield["TenantChange"] =  Mansfield['name'].shift(-1) != Mansfield['name'] 

In [730]:
Mansfield["TenantChange"].iloc[0] = False

In [731]:
MansfieldSub = Mansfield[(Mansfield['StatusChange'] == False)  & (Mansfield['TenantChange'] == True) & (Mansfield['status'] == True)]

In [732]:
counter = 0 
stoppingCondition = len(locations)
Mansfield2 = pd.DataFrame(data=None, columns=Mansfield.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Mansfield.loc[i, :].iloc[[ Mansfield.loc[i, :]['status_duration'].argmax()]]
        Mansfield2 = Mansfield2.append(m, ignore_index=False)
        counter += 1

In [733]:
Mansfield3 = MansfieldSub.append(Mansfield2)

In [734]:
Mansfield3 = Mansfield3.drop_duplicates()

In [735]:
Mansfield3.loc[Mansfield3['time-at-open'] < 4018, ['vacant']] = 1

In [736]:
Mansfield3['highstreet'] = 'Mansfield'

In [737]:
Mansfield3.to_csv('Mansfield-vacant-survival.csv')

#### 17. WestEnd

In [738]:
WestEnd = WestEnd.sort_values(by= ['status_date'])

In [739]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [740]:
#deleted those cols columns from the dataset as they only contain nulls

WestEnd = WestEnd.drop(cols, axis=1)

In [741]:
WestEnd['new_date'] = pd.to_datetime(WestEnd['status_date'], errors='coerce')

In [742]:
WestEnd = WestEnd.sort_values(by= ['new_date'])

In [743]:
WestEnd['new_date'].loc[(WestEnd['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [744]:
WestEnd = WestEnd.sort_values(by= ['location_code', 'period'])

In [745]:
WestEnd['locaton_code'] = WestEnd['location_code'].astype("category")

In [746]:
locations = WestEnd['location_code'].unique()

In [747]:
WestEnd.loc[WestEnd['name'].isnull(), 'name'] = WestEnd['location_code']

In [748]:
WestEnd['status_date'] = pd.to_datetime(WestEnd['status_date'], errors='coerce')

In [749]:
WestEnd['status_date'].iloc[WestEnd['status_date'] < '2010-04-01'] = '04/01/2010'

In [750]:
WestEnd['status_date'] = pd.to_datetime(WestEnd['status_date'], errors='coerce')

In [751]:
WestEnd['period2'] = pd.to_datetime(WestEnd['period'])

In [752]:
WestEnd['period2'] = WestEnd['period2'] + pd.DateOffset(months=1)

In [753]:
WestEnd['duration'] = WestEnd['period2'] - WestEnd['status_date']

In [754]:
WestEnd['duration'] = WestEnd['duration'].dt.days

In [755]:
WestEnd['study-start'] = '04/01/2010'

In [756]:
WestEnd['study-start'] = pd.to_datetime(WestEnd['study-start'], errors='coerce')

In [757]:
WestEnd['time-of-entry'] = WestEnd['status_date'] -  WestEnd['study-start']

In [758]:
WestEnd['time-at-open'] = 0

In [759]:
WestEnd['vacant'] = 0

In [760]:
WestEnd = WestEnd.drop(WestEnd[(WestEnd['status'] == False) & (WestEnd['status_date'] == '2010-04-01')].index)

In [761]:
WestEnd['location_code'] = WestEnd['location_code'].astype("category")

In [762]:
locations = list(WestEnd['location_code'].unique())

In [763]:
WestEnd = WestEnd.set_index('location_code')

In [764]:
WestEnd['survival'] = 0

In [765]:
WestEnd['survival'] = WestEnd['duration']

In [766]:
WestEnd['entry'] = WestEnd['time-of-entry'].dt.days

In [767]:
WestEnd['time-at-open'] = WestEnd['survival'] + WestEnd['entry']

In [768]:
WestEnd['stat'] = WestEnd['status'].astype(int)

In [769]:
WestEnd["StatusChange"] =  WestEnd['stat'].shift() != WestEnd['stat']

In [770]:
WestEnd["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [771]:
WestEnd["TenantChange"] =  WestEnd['name'].shift(-1) != WestEnd['name'] 

In [772]:
WestEnd["TenantChange"].iloc[0] = False

In [773]:
WestEndSub = WestEnd[(WestEnd['StatusChange'] == False)  & (WestEnd['TenantChange'] == True) & (WestEnd['status'] == True)]

In [774]:
counter = 0 
stoppingCondition = len(locations)
WestEnd2 = pd.DataFrame(data=None, columns=WestEnd.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = WestEnd.loc[i, :].iloc[[ WestEnd.loc[i, :]['status_duration'].argmax()]]
        WestEnd2 = WestEnd2.append(m, ignore_index=False)
        counter += 1

In [775]:
WestEnd3 = WestEndSub.append(WestEnd2)

In [776]:
WestEnd3 = WestEnd3.drop_duplicates()

In [777]:
WestEnd3.loc[WestEnd3['time-at-open'] < 4018, ['vacant']] = 1

In [778]:
WestEnd3['highstreet'] = 'WestEnd'

In [779]:
WestEnd3.to_csv('WestEnd-vacant-survival.csv')

#### 18. Hampstead

In [780]:
Hampstead = Hampstead.sort_values(by= ['status_date'])

In [781]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [782]:
#deleted those cols columns from the dataset as they only contain nulls

Hampstead = Hampstead.drop(cols, axis=1)

In [783]:
Hampstead['new_date'] = pd.to_datetime(Hampstead['status_date'], errors='coerce')

In [784]:
Hampstead = Hampstead.sort_values(by= ['new_date'])

In [785]:
Hampstead['new_date'].loc[(Hampstead['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [786]:
Hampstead = Hampstead.sort_values(by= ['location_code', 'period'])

In [787]:
Hampstead['locaton_code'] = Hampstead['location_code'].astype("category")

In [788]:
locations = Hampstead['location_code'].unique()

In [789]:
Hampstead.loc[Hampstead['name'].isnull(), 'name'] = Hampstead['location_code']

In [790]:
Hampstead['status_date'] = pd.to_datetime(Hampstead['status_date'], errors='coerce')

In [791]:
Hampstead['status_date'].iloc[Hampstead['status_date'] < '2010-04-01'] = '04/01/2010'

In [792]:
Hampstead['status_date'] = pd.to_datetime(Hampstead['status_date'], errors='coerce')

In [793]:
Hampstead['period2'] = pd.to_datetime(Hampstead['period'])

In [794]:
Hampstead['period2'] = Hampstead['period2'] + pd.DateOffset(months=1)

In [795]:
Hampstead['duration'] = Hampstead['period2'] - Hampstead['status_date']

In [796]:
Hampstead['duration'] = Hampstead['duration'].dt.days

In [797]:
Hampstead['study-start'] = '04/01/2010'

In [798]:
Hampstead['study-start'] = pd.to_datetime(Hampstead['study-start'], errors='coerce')

In [799]:
Hampstead['time-of-entry'] = Hampstead['status_date'] -  Hampstead['study-start']

In [800]:
Hampstead['time-at-open'] = 0

In [801]:
Hampstead['vacant'] = 0

In [802]:
Hampstead = Hampstead.drop(Hampstead[(Hampstead['status'] == False) & (Hampstead['status_date'] == '2010-04-01')].index)

In [803]:
Hampstead['location_code'] = Hampstead['location_code'].astype("category")

In [804]:
locations = list(Hampstead['location_code'].unique())

In [805]:
Hampstead = Hampstead.set_index('location_code')

In [806]:
Hampstead['survival'] = 0

In [807]:
Hampstead['survival'] = Hampstead['duration']

In [808]:
Hampstead['entry'] = Hampstead['time-of-entry'].dt.days

In [809]:
Hampstead['time-at-open'] = Hampstead['survival'] + Hampstead['entry']

In [810]:
Hampstead['stat'] = Hampstead['status'].astype(int)

In [811]:
Hampstead["StatusChange"] =  Hampstead['stat'].shift() != Hampstead['stat']

In [812]:
Hampstead["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [813]:
Hampstead["TenantChange"] =  Hampstead['name'].shift(-1) != Hampstead['name'] 

In [814]:
Hampstead["TenantChange"].iloc[0] = False

In [815]:
HampsteadSub = Hampstead[(Hampstead['StatusChange'] == False)  & (Hampstead['TenantChange'] == True) & (Hampstead['status'] == True)]

In [816]:
counter = 0 
stoppingCondition = len(locations)
Hampstead2 = pd.DataFrame(data=None, columns=Hampstead.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Hampstead.loc[i, :].iloc[[ Hampstead.loc[i, :]['status_duration'].argmax()]]
        Hampstead2 = Hampstead2.append(m, ignore_index=False)
        counter += 1

In [817]:
Hampstead3 = HampsteadSub.append(Hampstead2)

In [818]:
Hampstead3 = Hampstead3.drop_duplicates()

In [819]:
Hampstead3.loc[Hampstead3['time-at-open'] < 4018, ['vacant']] = 1

In [820]:
Hampstead3['highstreet'] = 'Hampstead'

In [821]:
Hampstead3.to_csv('Hampstead-vacant-survival.csv')

#### 19. Eversholt

In [822]:
Eversholt = Eversholt.sort_values(by= ['status_date'])

In [823]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [824]:
#deleted those cols columns from the dataset as they only contain nulls

Eversholt = Eversholt.drop(cols, axis=1)

In [825]:
Eversholt['new_date'] = pd.to_datetime(Eversholt['status_date'], errors='coerce')

In [826]:
Eversholt = Eversholt.sort_values(by= ['new_date'])

In [827]:
Eversholt['new_date'].loc[(Eversholt['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [828]:
Eversholt = Eversholt.sort_values(by= ['location_code', 'period'])

In [829]:
Eversholt['locaton_code'] = Eversholt['location_code'].astype("category")

In [830]:
locations = Eversholt['location_code'].unique()

In [831]:
Eversholt.loc[Eversholt['name'].isnull(), 'name'] = Eversholt['location_code']

In [832]:
Eversholt['status_date'] = pd.to_datetime(Eversholt['status_date'], errors='coerce')

In [833]:
Eversholt['status_date'].iloc[Eversholt['status_date'] < '2010-04-01'] = '04/01/2010'

In [834]:
Eversholt['status_date'] = pd.to_datetime(Eversholt['status_date'], errors='coerce')

In [835]:
Eversholt['period2'] = pd.to_datetime(Eversholt['period'])

In [836]:
Eversholt['period2'] = Eversholt['period2'] + pd.DateOffset(months=1)

In [837]:
Eversholt['duration'] = Eversholt['period2'] - Eversholt['status_date']

In [838]:
Eversholt['duration'] = Eversholt['duration'].dt.days

In [839]:
Eversholt['study-start'] = '04/01/2010'

In [840]:
Eversholt['study-start'] = pd.to_datetime(Eversholt['study-start'], errors='coerce')

In [841]:
Eversholt['time-of-entry'] = Eversholt['status_date'] -  Eversholt['study-start']

In [842]:
Eversholt['time-at-open'] = 0

In [843]:
Eversholt['vacant'] = 0

In [844]:
Eversholt = Eversholt.drop(Eversholt[(Eversholt['status'] == False) & (Eversholt['status_date'] == '2010-04-01')].index)

In [845]:
Eversholt['location_code'] = Eversholt['location_code'].astype("category")

In [846]:
locations = list(Eversholt['location_code'].unique())

In [847]:
Eversholt = Eversholt.set_index('location_code')

In [848]:
Eversholt['survival'] = 0

In [849]:
Eversholt['survival'] = Eversholt['duration']

In [850]:
Eversholt['entry'] = Eversholt['time-of-entry'].dt.days

In [851]:
Eversholt['time-at-open'] = Eversholt['survival'] + Eversholt['entry']

In [852]:
Eversholt['stat'] = Eversholt['status'].astype(int)

In [853]:
Eversholt["StatusChange"] =  Eversholt['stat'].shift() != Eversholt['stat']

In [854]:
Eversholt["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [855]:
Eversholt["TenantChange"] =  Eversholt['name'].shift(-1) != Eversholt['name'] 

In [856]:
Eversholt["TenantChange"].iloc[0] = False

In [857]:
EversholtSub = Eversholt[(Eversholt['StatusChange'] == False)  & (Eversholt['TenantChange'] == True) & (Eversholt['status'] == True)]

In [858]:
counter = 0 
stoppingCondition = len(locations)
Eversholt2 = pd.DataFrame(data=None, columns=Eversholt.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Eversholt.loc[i, :].iloc[[ Eversholt.loc[i, :]['status_duration'].argmax()]]
        Eversholt2 = Eversholt2.append(m, ignore_index=False)
        counter += 1

In [859]:
Eversholt3 = EversholtSub.append(Eversholt2)

In [860]:
Eversholt3 = Eversholt3.drop_duplicates()

In [861]:
Eversholt3.loc[Eversholt3['time-at-open'] < 4018, ['vacant']] = 1

In [862]:
Eversholt3['highstreet'] = 'Eversholt'

In [863]:
Eversholt3.to_csv('Eversholt-vacant-survival.csv')

****

#### Append all for Camden total survival

In [870]:
camden_survival = pd.concat([Brecknock3, Kentish3, Fortress3, CamdenT3, Belsize3, Loudoun3, Haverstock3, SouthEnd3, Cricklewood3, Regents3, Queens3, Kilburn3, Mill3, Fortune3, Willesden3, Mansfield3, WestEnd3, Hampstead3, Eversholt3], ignore_index=True)

In [871]:
camden_survival.to_csv('camden-vacant-survival-full.csv')